## Install and Import Dependencies

In [12]:
pip install tensorflow numpy matplotlib IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.7 MB/s eta 0:00:00


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, ReLU, Conv2DTranspose, Conv2D, LeakyReLU, Dropout, Flatten
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from IPython import display

## Load Dataset

In [2]:
# Load and preprocess Fashion MNIST dataset
(train_images, _), (_, _) = fashion_mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize to [-1, 1]

BUFFER_SIZE = 60000
BATCH_SIZE = 256
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Generator Model Architecture

In [13]:
generator = Sequential([
    Dense(7 * 7 * 256, use_bias = False, input_shape = (100,)),
    BatchNormalization(),
    ReLU(),
    Reshape((7, 7, 256)),
    Conv2DTranspose(128, (5, 5), strides = (1, 1), padding = 'same', use_bias = False),
    BatchNormalization(),
    ReLU(),
    Conv2DTranspose(64, (5, 5), strides = (2, 2), padding = 'same', use_bias = False),
    BatchNormalization(),
    ReLU(),
    Conv2DTranspose(1, (5, 5), strides = (2, 2), padding = 'same', use_bias = False, activation = 'tanh')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Discriminator Model Architecture

In [14]:
discriminator = Sequential([
    Conv2D(64, (5, 5), strides = (2, 2), padding = 'same', input_shape = [28, 28, 1]),
    LeakyReLU(),
    Dropout(0.3),
    Conv2D(128, (5, 5), strides = (2, 2), padding = 'same'),
    LeakyReLU(),
    Dropout(0.3),
    Flatten(),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Defining Losses

In [6]:
cross_entropy = BinaryCrossentropy(from_logits = True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

## Defining Optimizers

In [7]:
generator_optimizer = Adam(1e-4)
discriminator_optimizer = Adam(1e-4)

## Parameters for Training

In [8]:
# Training Loop
EPOCHS = 150
NOISE_DIM = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, NOISE_DIM])

def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training = True)
        real_output = discriminator(images, training = True)
        fake_output = discriminator(generated_images, training = True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss

## Generating Images

In [9]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training = False)
    fig = plt.figure(figsize = (4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap = 'gray')
        plt.axis('off')
    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.close()

## Model Training

In [10]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        gen_losses, disc_losses = [], []
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)
        avg_gen_loss = tf.reduce_mean(gen_losses)
        avg_disc_loss = tf.reduce_mean(disc_losses)
        print(f'Epoch {epoch+1}, Gen Loss: {avg_gen_loss:.4f}, Disc Loss: {avg_disc_loss:.4f}, Time: {time.time()-start:.2f} sec')
        if (epoch + 1) % 5 == 0:
            generate_and_save_images(generator, epoch + 1, seed)
    return generator, discriminator

#### Fitting Model

In [11]:
generator, discriminator = train(train_dataset, EPOCHS)
generator.save('fashion_mnist_generator.keras')
discriminator.save('fashion_mnist_discriminator.keras')

Epoch 1, Gen Loss: 0.7277, Disc Loss: 1.0730, Time: 58.38 sec
Epoch 2, Gen Loss: 1.0323, Disc Loss: 1.0704, Time: 81.91 sec
Epoch 3, Gen Loss: 0.9985, Disc Loss: 1.1715, Time: 51.03 sec
Epoch 4, Gen Loss: 1.0591, Disc Loss: 1.1022, Time: 49.68 sec
Epoch 5, Gen Loss: 0.9618, Disc Loss: 1.2184, Time: 50.40 sec
Epoch 6, Gen Loss: 0.8483, Disc Loss: 1.2545, Time: 50.00 sec
Epoch 7, Gen Loss: 0.8451, Disc Loss: 1.2498, Time: 49.19 sec
Epoch 8, Gen Loss: 0.9003, Disc Loss: 1.2288, Time: 81.91 sec
Epoch 9, Gen Loss: 0.8512, Disc Loss: 1.3370, Time: 81.91 sec
Epoch 10, Gen Loss: 0.9234, Disc Loss: 1.2051, Time: 50.53 sec
Epoch 11, Gen Loss: 0.9697, Disc Loss: 1.1875, Time: 50.45 sec
Epoch 12, Gen Loss: 0.9370, Disc Loss: 1.1741, Time: 50.58 sec
Epoch 13, Gen Loss: 1.0055, Disc Loss: 1.1235, Time: 49.70 sec
Epoch 14, Gen Loss: 1.0162, Disc Loss: 1.1406, Time: 49.57 sec
Epoch 15, Gen Loss: 1.0155, Disc Loss: 1.1042, Time: 49.56 sec
Epoch 16, Gen Loss: 0.9856, Disc Loss: 1.1806, Time: 81.91 sec
E